<a href="https://colab.research.google.com/github/CodeMonkey01/DataMiningI/blob/main/ANN/Option_A/ANN-FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd '/content/drive/MyDrive/'

    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/MyDrive
Thu May 26 19:25:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [3]:
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 7.0 MB/s 
     |████████████████████████████████| 511.7 MB 6.8 kB/s 
     |████████████████████████████████| 1.6 MB 36.4 MB/s 
     |████████████████████████████████| 5.8 MB 39.5 MB/s 
     |████████████████████████████████| 438 kB 51.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Data Mining/dataset.txt')
df.describe()

,text,humor
count,200000,200000
unique,200000,2
top,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
freq,1,100000


In [6]:
# Transform class from Boolean to integer value
df['class']=df['humor'].apply(lambda x: 1 if x==True else 0)

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

MAX_LEN = 128
#pad_to_max_length=True,
df['bert_preprocessed']=df['text'].apply(lambda x: " ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(x, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))))

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
df.head()

,text,humor,class,bert_preprocessed
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,0,[CLS] Joe bid ##en rules out 2020 bid : ' guys...
1,Watch: darvish gave hitter whiplash with slow ...,False,0,[CLS] Watch : da ##r ##vis ##h gave hitter whi...
2,What do you call a turtle without its shell? d...,True,1,[CLS] What do you call a turtle without its sh...
3,5 reasons the 2016 election feels so personal,False,0,[CLS] 5 reasons the 2016 election feels so per...
4,"Pasco police shot mexican migrant from behind,...",False,0,[CLS] Pa ##sco police shot me ##xi ##can migra...


In [9]:
X = df['bert_preprocessed']
Y = df['class']

In [10]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4")

In [11]:
def build_model() -> tf.keras.Model:
    # Bert layers
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Neural network layers
    l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

    # Use inputs and outputs to construct a final model
    model = tf.keras.Model(inputs=[text_input], outputs = [l])

    #model.summary()

    return model

In [ ]:
# todo
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['bert_preprocessed'], df['class'], test_size=0.4)

import numpy as np
import itertools
EPOCHS = 5 # reduce to 3 EPOCHS to safe run time (1 epoch varies from 25 minutes to 1 hour)

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]

LEARNING_RATE = 0.007
EPSILON = 1e-06

batch_values = [256]
for batch in batch_values:
  model: tf.keras.Model = build_model()

  print(f"Try batch size: {batch}")

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS)
    
  model.fit(X_train, y_train, epochs=EPOCHS, batch_size=batch)
  model.evaluate(X_test, y_test)

Try batch size: 256
Epoch 1/5
469/469 [==============================] - 1805s 4s/step - loss: 0.4033 - accuracy: 0.8255 - precision: 0.8234 - recall: 0.8278
Epoch 2/5
410/469 [=========================>....] - ETA: 3:42 - loss: 0.2875 - accuracy: 0.8842 - precision: 0.8832 - recall: 0.8848

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

ADAM_LEARNING_RATE = 0.0044
ADAM_EPSILON = 1e-05

EPOCH = 50

BATCH_SIZE = 64

PATIENCE = 2

base_path = "/content/drive/MyDrive/Data Mining/bert/"

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]

def train_evaluate(model, run, x_train, y_train, x_test, y_test):
    STAMP = 'model_run_%.2f'%(run)
    early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE)
    bst_model_path = base_path + STAMP + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=ADAM_LEARNING_RATE, epsilon=ADAM_EPSILON),
                  loss=tf.keras.losses.BinaryCrossentropy(),                  
                  metrics=METRICS)
    

    X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, test_size=0.2)

    hist = model.fit(X_tra, y_tra, validation_data=(X_val, y_val), epochs=EPOCH, batch_size=BATCH_SIZE, shuffle=True, callbacks=[early_stopping, model_checkpoint])

    return model.evaluate(x_test, y_test)

kFold = StratifiedKFold(n_splits=4)
accuracy_list = []
precision_list = []
recall_list = []
i = 1
for train, test in kFold.split(X, Y):
    model = build_model()
    _, accuracy, precision, recall = train_evaluate(model, i, X[train], Y[train], X[test], Y[test])
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    i = i + 1

print(f"Accuracy: {sum(accuracy_list) / len(accuracy_list)}")
print(f"Precision: {sum(precision_list) / len(precision_list)}")
print(f"Recall: {sum(recall_list) / len(recall_list)}")

In [ ]:
# model.save_weights("/content/drive/MyDrive/Data Mining/bert_seperator/weights.md5")

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

# Own test

In [ ]:
jokes = [
    'What’s the best thing about Switzerland? I don’t know, but the flag is a big plus.',
    'I study Business Informatics at the University of Mannheim!',
    'I invented a new word! Plagiarism!',
    'Did you hear about the mathematician who’s afraid of negative numbers? He’ll stop at nothing to avoid them.',
    'My name is Elias.'
]

jokes_processed = [" ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(i, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))) for i in jokes]

model.predict(jokes_processed)